In [1]:
import pandas as pd

In [2]:
!curl https://raw.githubusercontent.com/tvthanh0401/Recommender-System/main/SampleData/ratings.csv --output ratings.csv
!curl https://raw.githubusercontent.com/tvthanh0401/Recommender-System/main/SampleData/movies.csv --output movies.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6696k  100 6696k    0     0  13.6M      0 --:--:-- --:--:-- --:--:-- 13.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  133k  100  133k    0     0   523k      0 --:--:-- --:--:-- --:--:--  523k


In [3]:
movies = pd.read_csv('movies.csv', encoding='latin')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings = pd.read_csv('ratings.csv')

In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


 Một phương án có thể đề xuất Non-Personalized đó là gợi ý dựa trên rating trung bình của từng bộ phim, khi đó chúng ta sẽ chọn ra những bộ phim được đánh giá cao nhất.

In [ ]:
# Tính tổng rating của bộ phim.
sumRatingPerItem = ratings.groupby('movieId')['rating'].sum()

In [ ]:
sumRatingPerItem.head()

movieId
1    1881.5
2     856.5
3     228.0
4      33.5
5     216.5
Name: rating, dtype: float64

In [ ]:
# Tính số lượt được đánh giá của từng bộ phim
countPerItem = ratings.groupby('movieId')['rating'].count()

In [ ]:
# Tính rating trung bình, sau đó sắp xếp theo thứ tự giảm dần.
sortedItem = (sumRatingPerItem / countPerItem).sort_values(ascending=False)

In [ ]:
type(sortedItem)

pandas.core.series.Series

In [ ]:
recommendItem = sortedItem.iloc[:10].to_dict()

In [ ]:
# Top 10 bộ phim có rating trung bình cao nhất.
recommendItem.keys()

dict_keys([318, 858, 1248, 2959, 7502, 1203, 2859, 1221, 296, 2571])

 Đối với phương án trên, sẽ có vấn đề nảy sinh, đó là chẳng hạn có một bộ phim $i$ chỉ được đánh giá 1 lần với giá trị rating là 5 chẳng hạn. Khi đó bộ phim $i$ này hiển nhiên sẽ nằm trong danh sách đề xuất. Tuy nhiên với số lần đánh giá thấp như vậy thì chưa chắc bộ phim $i$ đã thực sự hay mà đơn giản nó chỉ phù hợp với 1 số đối tượng nhất định và hoàn toàn có thể bộ phim thứ $i$ này sẽ bị đánh giá thấp trong những lượt đánh giá tiếp theo. Để khắc phục tình trạng trên thì ta sẽ sử dụng một kỹ thuật gọi là **damping mean**, ta sẽ thêm vào cho mỗi bộ phim 1 lượng rating "ảo" với mức đánh giá là rating trung bình của toàn bộ dataset, ta dễ thấy khi số lượng rating càng nhiều thì rating của các bộ phim sẽ không còn chịu ảnh hưởng của giá trị damping này nữa.

In [ ]:
# Khởi tạo giá trị damping
damping = 5

In [ ]:
globalSumRating = ratings['rating'].sum()
globalCountRating = ratings['rating'].count()
print(globalSumRating, globalCountRating)

926842.0 264505


In [ ]:
# Rating trung bình của toàn bộ dataset
alpha = globalSumRating / globalCountRating
print(alpha)

3.504062305060396


In [ ]:
# Tính tổng rating + phần rating ảo thêm vào
dampingSumRatingPerIten = sumRatingPerItem + damping * alpha

In [ ]:
# Tính số lượt đánh giá + số lượt đánh giá ảo được thêm vào
dampingCountPerItem = countPerItem + damping

In [ ]:
sortDampingItem = (dampingSumRatingPerIten / dampingCountPerItem).sort_values(ascending = False)

In [ ]:
dampingRecommendItem = sortDampingItem.iloc[:10].to_dict()

In [ ]:
# Các bộ phim được đề xuất dựa trên damping- mean, ở đây chúng ta sẽ thấy một số
# phim đã không còn xuất hiện trong danh sách đề xuất.
dampingRecommendItem.keys()

dict_keys([318, 858, 2959, 1203, 296, 7502, 1221, 1248, 2571, 4226])

In [ ]:
referenceItem = 260

In [ ]:
candidate = ratings[ratings['movieId'] == referenceItem]

In [ ]:
len(candidate)

535

In [ ]:
userList = list(candidate['userId'].values)

In [ ]:
ratingList = ratings[ratings['userId'].isin(userList)]

In [ ]:
set(ratingList['userId'].values) == set(userList)

True

In [ ]:
referenceItemCount = ratingList.groupby('movieId')['rating'].count()

In [ ]:
referenceItemCount.max()

535

In [ ]:
confidence = (referenceItemCount / referenceItemCount[referenceItem]).sort_values(ascending = False)

In [ ]:
confidence[1:10].to_dict().keys()

dict_keys([2571, 1196, 4993, 1210, 356, 5952, 7153, 296, 1198])

Đề xuất sử dụng **association rule**:
- Ta thường gặp đề xuất dạng: "Những người đã xem bộ phim **A** thì họ thường xem bộ phim **B**. Ở đây ta sẽ xây dựng các đề xuất dựa trên những luật như vậy dựa trên 2 giá trị là confidence và lift.
- Confidence({A}->{B}) = $P(B | A)$ được tính như sau:
$$ P(B | A) = \frac{P(A \cap B)}{P(A)}$$ 
- Lift({A} -> {B}):
$$s(B|A) = \frac {P(A \cap B)} {P(A) P(B)}$$

In [69]:
def recommendByConfidence(ratings, referenceItem, topK=5):
  userList = list(ratings[ratings['movieId'] == referenceItem]['userId'].values)
  ratingSubset = ratings[ratings['userId'].isin(userList)]
  intersectItemCount = ratingSubset.groupby('movieId')['rating'].count()
  confidence = (intersectItemCount / intersectItemCount[referenceItem]).sort_values(ascending = False)
  return list(confidence[1: topK + 1].to_dict().keys())

In [70]:
recommendByConfidence(ratings, 260)

[2571, 1196, 4993, 1210, 356]

In [72]:
def recommendByLift(ratings, referenceItem, topK=5):
  userList = list(ratings[ratings['movieId'] == referenceItem]['userId'].values)
  ratingIntersectSubset = ratings[ratings['userId'].isin(userList)]
  intersectItemCount = ratingIntersectSubset.groupby('movieId')['rating'].count()
  movieList = list(set(ratingIntersectSubset['movieId'].values))
  ratingCandidateSubset = ratings[ratings['movieId'].isin(movieList)]
  itemCount = ratingCandidateSubset.groupby('movieId')['rating'].count()
  total = len(set(ratings['userId'].values))
  print(total)
  lift = (intersectItemCount * total / (itemCount * intersectItemCount[referenceItem])).sort_values(ascending = False)
  print(lift[1:topK + 1])
  return list(lift[1: topK + 1].to_dict().keys())

In [73]:
recommendByLift(ratings, 2761)

862
movieId
631     4.897727
2532    4.810268
3615    4.545703
2439    4.489583
1016    4.489583
Name: rating, dtype: float64


[631, 2532, 3615, 2439, 1016]